In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub

In [2]:
splits, info = tfds.load("horses_or_humans",
                         as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'],
                         data_dir="./data")

train_examples, validation_examples, test_examples = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes
num_examples, num_classes

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1027 [00:00<?, ? examples/s]

Shuffling data/horses_or_humans/3.0.0.incompleteRDYFO2/horses_or_humans-train.tfrecord*...:   0%|          | 0…

Generating test examples...:   0%|          | 0/256 [00:00<?, ? examples/s]

Shuffling data/horses_or_humans/3.0.0.incompleteRDYFO2/horses_or_humans-test.tfrecord*...:   0%|          | 0/…

Dataset horses_or_humans downloaded and prepared to ./data/horses_or_humans/3.0.0. Subsequent calls will reuse this data.


(1027, 2)

In [3]:
BATCH_SIZE = 32
IMAGE_SIZE = 224

In [4]:
# Create a autograph pre-processing function to resize and normalize an image
### START CODE HERE ###
@tf.function
def map_fn(image, label):
  img = tf.image.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
  img /= 255.
  return img, label

In [5]:
## TEST CODE:

test_image, test_label = list(train_examples)[0]

test_result = map_fn(test_image, test_label)

print(test_result[0].shape)
print(test_result[1].shape)

del test_image, test_label, test_result

(224, 224, 3)
()


In [6]:
train_ds = train_examples.map(map_fn).shuffle(buffer_size=1024).batch(BATCH_SIZE)
valid_ds = validation_examples.map(map_fn).batch(BATCH_SIZE)
test_ds = test_examples.map(map_fn).batch(BATCH_SIZE)

In [8]:
## TEST CODE:

test_train_ds = list(train_ds)
print(len(test_train_ds))
print(test_train_ds[0][0].shape)

del test_train_ds

26
(32, 224, 224, 3)


In [7]:
MODULE_HANDLE = "https://www.kaggle.com/models/tensorflow/resnet-50/TensorFlow2/feature-vector/1"
model = tf.keras.Sequential([
    hub.KerasLayer(MODULE_HANDLE, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    tf.keras.layers.Dense(num_classes, activation="softmax")
])

In [9]:
def set_adam_optimizer():
  optimizer = tf.keras.optimizers.Adam()
  return optimizer

In [10]:
## TEST CODE:

test_optimizer = set_adam_optimizer()

print(type(test_optimizer))

del test_optimizer

<class 'keras.src.optimizers.adam.Adam'>


In [11]:
def set_sparse_cat_crossentropy_loss():
  train_loss = tf.keras.losses.SparseCategoricalCrossentropy()
  val_loss = tf.keras.losses.SparseCategoricalCrossentropy()
  return train_loss, val_loss

In [12]:
## TEST CODE:

test_train_loss, test_val_loss = set_sparse_cat_crossentropy_loss()

print(type(test_train_loss))
print(type(test_val_loss))

del test_train_loss, test_val_loss

<class 'keras.src.losses.SparseCategoricalCrossentropy'>
<class 'keras.src.losses.SparseCategoricalCrossentropy'>


In [13]:
def set_sparse_cat_crossentropy_accuracy():
  train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
  val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
  return train_accuracy, val_accuracy

In [14]:
## TEST CODE:

test_train_accuracy, test_val_accuracy = set_sparse_cat_crossentropy_accuracy()

print(type(test_train_accuracy))
print(type(test_val_accuracy))

del test_train_accuracy, test_val_accuracy

<class 'keras.src.metrics.accuracy_metrics.SparseCategoricalAccuracy'>
<class 'keras.src.metrics.accuracy_metrics.SparseCategoricalAccuracy'>


In [15]:
optimizer = set_adam_optimizer()
train_loss, val_loss = set_sparse_cat_crossentropy_loss()
train_accuracy, val_accuracy = set_sparse_cat_crossentropy_accuracy()

In [16]:
EPOCHS = 2
def train_one_step(model, optimizer, x, y, train_loss, train_accuracy):
  with tf.GradientTape() as tape:
    y_pred = model(x)
    loss = train_loss(y, y_pred)

  gradients = tape.gradient(loss, model.trainable_weights)
  optimizer.apply_gradients(zip(gradients, model.trainable_weights))

  train_accuracy(y, y_pred)
  return loss

In [17]:
## TEST CODE:

def base_model():
    inputs = tf.keras.layers.Input(shape=(2))
    x = tf.keras.layers.Dense(64, activation='relu')(inputs)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

test_model = base_model()

test_optimizer = set_adam_optimizer()
test_image = tf.ones((2,2))
test_label = tf.ones((1,))
test_train_loss, _ = set_sparse_cat_crossentropy_loss()
test_train_accuracy, _ = set_sparse_cat_crossentropy_accuracy()

test_result = train_one_step(test_model, test_optimizer, test_image, test_label, test_train_loss, test_train_accuracy)
print(test_result)

del test_result, test_model, test_optimizer, test_image, test_label, test_train_loss, test_train_accuracy

tf.Tensor(0.6931472, shape=(), dtype=float32)


In [ ]:
@tf.function
def